# Molecular docking

Aquest notebook inclou la preparació dels fitxers, docking i analisis de les poses obtingudes.

Aquest notebook ha estat modificat de https://github.com/AngelRuizMoreno/Jupyter_Dock

## Content of this notebook

1. Afegir protons / Preparació dels inputs
2. Visualització 
3. Docking utilitzant el programa LeDock
    - Receptor preparation
    - Ligand preparation
    - Docking box definition
    - Docking
    - DOK results file conversion to SDF
    - 3D visualization of docking results
    - 2D interaction table and map

# Importem totes les dependencies necesaries

- openbabel s'utilitza per cambiar de format els fitxers, ja que cada programa requereix d'un format específic. Llavors neceitarém transformar els fitxers per al seu posterior analisis
- rdkit, MDAnalysis i prolif son llibreries que permeten la interpretació i anàlisis de molécules. Amb aquestes llibreries carreguem els resultats i analitzem les interaccions
- pymol s'utilitza per a la visualització

- El programa de docking i preparació dels imputs (Afegir protons), son exacutables que no s'han de tenir importats, sinó que cal tenir en l'ordinador on s'executa

In [3]:
from pymol import cmd
import py3Dmol

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork


import sys, os
sys.path.insert(1, 'utilities/')
from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf


import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

## 1. Afegir protons / Preparació dels inputs

In [4]:
os.chdir('Clara_TDR/') #Anar a la carpeta asignada pel projecte

#### Method 1: LePro

El programa de Docking inclou un executable que ens prepara la proteina per al docking. Aixó vol dir que afegeix els protons que no es poden resoldre en la cristalització.

Per utilitzar l'executable, es fa com si fos la consola de comandes. Per indicar que el seguent codi s'ha de executar en consola, es pot utilitzar el símbol de exclamació (!). Aquesta és una funcionalitat de jupyter notebooks. En cas de voler executar una comanda en un script .py o similar, recomano la utilització de la comanda os.system()

In [2]:
!../bin/lepro_linux_x86 # Launch this cell to see parameters

/bin/bash: ../bin/lepro_linux_x86: No such file or directory


Executem el programa que ens protonarà i després anomenen al fitxer que obtenim.
Aixó ho fem pels nostres 3 sistemes

In [9]:
!../bin/lepro_linux_x86 HG1.pdb

os.rename('pro.pdb','HG1_H.pdb') # Output from lepro is pro.pdb, this line will change the name to '1AZ8_clean_H.pdb'

In [22]:
!../bin/lepro_linux_x86 HG3.pdb

os.rename('pro.pdb','HG3_H.pdb') # Output from lepro is pro.pdb, this line will change the name to '1AZ8_clean_H.pdb'

In [23]:
!../bin/lepro_linux_x86 HG3.17.pdb

os.rename('pro.pdb','HG3.17_H.pdb') # Output from lepro is pro.pdb, this line will change the name to '1AZ8_clean_H.pdb'

## 2. System Visualization

En aquest pas, podem visualitzar la proteina amb el nostre estat de transcisió per aixó poder asegurar-nos de que està tot correctament.

Obiament aquest pas és completament opcional

In [5]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('HG1_H.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('ts.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 3. Docking with LeDock

#### Ara podem fer el docking utilitzant el programa de LeDock. S'ha seleccionat aquest programa per multiples raons. Donat a que dona bons resultats, permet flexibilitat dels residus, és un sol executable, i requereix de que els fitxers estiguin en un format específic com PDBQT, com si que necesita tots els programes basats en autodock (Vina, smina,...)

LeDock is designed for fast and accurate flexible docking of small molecules into a protein. It achieves a pose-prediction accuracy of greater than 90% on the Astex diversity set and takes about 3 seconds per run for a drug-like molecule. It has led to the discovery of novel kinase inhibitors and bromodomain antagonists from high-throughput virtual screening campaigns. It directly uses SYBYL Mol2 format as input for small molecules.

### 3.1. Receptor preparation

In LeDock, the input file for a receptor is a PDB file with explicit hydrogens in all residues. LePro was created as a tool for preparing protein structures for docking with LeDock. Thus, at this stage, we can use the file ater sanitization steps as well as a protein structure from Jupyter Dock's _**fix_protein()**_ function.

### 3.2. Ligand preparation

As previously stated, the input format for ligand in LeDock is MOL2. Similarly to the receptor preparation, we can use the ligand directly after sanitization. 

### 3.3. Docking box definition

This step can be completed in the same manner as the AutoDock Vina box definition. To obtain the identical box from AutoDock Vina docking but in LeDock format, the user only needs to change the parameter "software" from "vina" to "ledock." 

>**Info:** The implementation of the _**get_box()**_ function allows for the easy replication of binding sites between AutoDock Vina and LeDock, with the goal of replicating and comparing results between both programs.

#### Llavors, cal generar la caixa o zona on farém el docking. En aquest cas, com que tenim un cristall de l'anàleg de l'estat de trascisió (TS) alineat amb la proteina, agafem aquest com a referencia i generem una caixa extenent 5Å cap a cada costat

In [16]:
cmd.load(filename='HG1_H.pdb',format='pdb',object='prot')
cmd.load(filename='ts.mol2',format='mol2',object='lig')

X,Y,Z= getbox(selection='lig',extending=5.0,software='ledock')
cmd.delete('all')

print(X,'\n',Y,'\n',Z)

{'minX': -15.470999717712402, 'maxX': 0.8289999961853027} 
 {'minY': 23.489999771118164, 'maxY': 37.71500015258789} 
 {'minZ': -3.621000051498413, 'maxZ': 9.302000045776367}


### 3.4. Docking

To run LeDock, a configuration (commonly known as dock.in) file containing all docking parameters as well as information about the receptor and ligand(s) to dock is required. Jupyter Dock uses the function _**generate ledock file()**_ to generate the configuration file automatically. After configuring the parameters, the user will receive a configuration file as well as a file containing a list of ligand(s) to dock (commonly named ligand.list). After this, docking would be as simple as launching the LeDock executable and the configuration file as parameter.

**_generate_ledock_file( params )_**

params:

- **receptor**: _str or path-like string_ ; protein file for docking including hydrogens, format must be pdb 
- **x**: _2 element list of floats [ float , float ]_; Xmin and Xmax coordinates of docking box
- **y**:  _2 element list of floats [ float , float ]_; Ymin and Ymax coordinates of docking box
- **z**:  _2 element list of floats [ float , float ]_; Zmin and Zmax coordinates of docking box
- **n_poses**: _float_ ; n_of poses to retrieve from docking 
- **rmsd**: _float_ ; minimum RMSD diference between docking poses 
- **l_list**: _list of n strings or path-like strings [ lig1, lig2, lig3 ... ] ; list of ligands or ligands paths to dock 
- **l_list_outfile**: _str or path-like string_ ; filename to save the ligand list, needed for ledock to locate ligands
- **out**: _str or path-like string_ ; outfile to save docking paramemeters, needed to launch the docking

Fem el docking

In [17]:
generate_ledock_file(receptor='HG1_H.pdb',x=[X['minX'],X['maxX']],
                     y=[Y['minY'],Y['maxY']],
                     z=[Z['minZ'],Z['maxZ']],
                     n_poses=10,
                     rmsd=1.0,
                     l_list='lig.mol2', 
                     l_list_outfile='ledock_ligand.list',
                     out='dock.in')

In [18]:
!../bin/ledock_linux_x86 #Launch this cell to see parameters

************************************************************
*       LeDock v1.0                                        *
*       Molecular Docking Software                         *
*       Copyright 2013-14 (C) H. Zhao PhD                  *
*       For academic use only                              *
*       www.lephar.com                                     *
************************************************************
--------Usage:
--------ledock config.file    !docking
--------ledock -spli dok.file !split into separate coordinates



In [19]:
!../bin/ledock_linux_x86 {'dock.in'}

### 3.5. DOK results file conversion to SDF

##### Per al anàlisi dels docking, cal cambiar el fitxer obtingut a un format diferent.


The end result of LeDock is a file with the dok extension that contains the docking properties in the same way that a pdb file does. Nonetheless, the dok file is not a widely used format for representing chemical structures. As a result, Jupyter Dock can convert dok files to the widely used sdf format. Jupyter Dock, like in the pdbqt to sdf conversion, will preserve the chemical features and save the "Pose" and "Score" results as molecule attributes.

**_dok_to_sdf ( params )_**

params:

   - **dok_file**: _str or path-like_ ; dok file from ledock docking

   - **output**: _str or path-like_ ; out file from ledock docking, extension must be sdf

In [21]:
dok_to_sdf(dok_file='lig.dok',output='HG1_dock.sdf')

*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

### 3.6. 3D visualization of docking results

###### Visualitzem el resultat i obtenim el "SCORE" obtingut

As with the system visualization, the docking results can be inspected and compared to the reference structure (if exist). The ligand's "Pose" and "Score" information will also be displayed to show how acces to this molecule's attributes.

In [6]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('HG1_H.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('ts.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

results=Chem.SDMolSupplier('HG1_dock.sdf')


p=Chem.MolToMolBlock(results[0])

print('Reference: Magenta | LeDock Pose: Cyan')
print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

Reference: Magenta | LeDock Pose: Cyan
Pose: 3 | Score: -3.23


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#-------------------------------------------------------------

### 3.7. 2D interaction table and map

###### Calculem la taula de interaccions i obtenim mapa de interaccions 2D

Inspecting the molecular interactions is one of the most common analyses performed by a computational scientist following a docking experiment. As a result, Jupyter Dock uses ProLif to create a table of ligand-protein molecular interactions as well as the corresponding 2D map.

> **Info:** ProLif uses RDKit and MDAnalysis to map the molecular interaction between a ligand and a protein. As a result, some protein preparations, including the use of LePro, can result in errors that impede analysis. Jupyter Dock's _fix protein()_ function can be used to avoid such errors and provide a suitable protein structure for the analysis.

In [17]:
fix_protein(filename='HG1.pdb',addHs_pH=7.4,try_renumberResidues=True,output='HG1_fix.pdb')

In [8]:
# load protein
prot = mda.Universe("HG1_fix.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

# load ligands
path = str('HG1_dock.sdf')
lig_suppl = list(plf.sdf_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
results_df = fp.to_dataframe(return_atoms=True)
results_df

  0%|          | 0/6 [00:00<?, ?it/s]

ligand               UNL1                                            \
protein          ASN130.A      ASN172.A      GLN207.A      GLU131.A   
interaction   Hydrophobic    HBAcceptor   Hydrophobic      Cationic   
Frame                                                                 
0                 (11, 9)  (None, None)      (11, 12)  (None, None)   
1            (None, None)  (None, None)  (None, None)  (None, None)   
2            (None, None)  (None, None)       (6, 12)  (None, None)   
3                 (10, 9)  (None, None)       (6, 12)       (4, 13)   
4            (None, None)       (1, 12)       (8, 12)  (None, None)   
5            (None, None)  (None, None)  (None, None)       (4, 13)   

ligand                                                             \
protein                        GLU237.A                  HIS209.A   
interaction   Hydrophobic      Cationic Hydrophobic    HBAcceptor   
Frame                                                               
0            (None, None)       (4, 13)     (5, 12)  (None, None)   
1                 (5, 12)       (4, 13)     (8, 12)       (2, 16)   
2                 (6, 12)       (4, 13)     (5, 12)  (None, None)   
3                 (5, 12)  (None, None)     (6, 12)  (None, None)   
4                (10, 12)  (None, None)     (5, 12)  (None, None)   
5                 (6, 12)  (None, None)     (5, 12)  (None, None)   

ligand                                                                        \
protein                       HIS83.A                  LYS50.A                 
interaction    PiStacking Hydrophobic PiStacking       Anionic    HBAcceptor   
Frame                                                                          
0            (None, None)     (5, 14)     (5, 9)       (1, 18)  (None, None)   
1            (None, None)     (6, 14)     (5, 9)  (None, None)  (None, None)   
2            (None, None)     (5, 14)     (5, 9)       (1, 18)  (None, None)   
3                  (0, 9)     (9, 14)     (5, 9)  (None, None)       (1, 19)   
4            (None, None)     (5, 14)     (5, 9)  (None, None)  (None, None)   
5            (None, None)     (6, 14)     (0, 9)  (None, None)  (None, None)   

ligand                                                             \
protein                                     MET21.A       MET46.A   
interaction Hydrophobic      PiCation   Hydrophobic   Hydrophobic   
Frame                                                               
0               (5, 15)       (5, 18)       (7, 12)  (None, None)   
1               (6, 15)  (None, None)  (None, None)  (None, None)   
2               (5, 15)  (None, None)       (5, 13)  (None, None)   
3               (8, 15)  (None, None)  (None, None)  (None, None)   
4               (5, 15)       (0, 18)       (5, 12)       (11, 2)   
5               (6, 15)  (None, None)  (None, None)  (None, None)   

ligand                                                 
protein           PHE89.A       TYR90.A                
interaction   Hydrophobic   Hydrophobic    PiStacking  
Frame                                                  
0            (None, None)       (9, 14)  (None, None)  
1                 (11, 6)       (5, 14)        (0, 9)  
2            (None, None)      (10, 14)  (None, None)  
3            (None, None)       (7, 14)  (None, None)  
4            (None, None)  (None, None)  (None, None)  
5                  (9, 6)       (6, 14)        (5, 9)

In [9]:
net = LigNetwork.from_ifp(results_df,lig_suppl[0],kind="frame", frame=0,rotation=270)
net.save('HG1_interaccions.html')
net.display()

## Ho repetim tot per la resta de sistemes

In [11]:
P ='HG3_H.pdb'

cmd.load(filename=P,format='pdb',object='prot')
cmd.load(filename='ts.mol2',format='mol2',object='lig')

X,Y,Z= getbox(selection='lig',extending=5.0,software='ledock')
cmd.delete('all')

print(X,'\n',Y,'\n',Z)

generate_ledock_file(receptor=P,x=[X['minX'],X['maxX']],
                     y=[Y['minY'],Y['maxY']],
                     z=[Z['minZ'],Z['maxZ']],
                     n_poses=10,
                     rmsd=1.0,
                     l_list='lig.mol2', 
                     l_list_outfile='ledock_ligand.list',
                     out='dock.in')

!../bin/ledock_linux_x86 {'dock.in'}

dok_to_sdf(dok_file='lig.dok',output='HG3_dock.sdf')

 PyMOL not running, entering library mode (experimental)
{'minX': -15.470999717712402, 'maxX': 0.8289999961853027} 
 {'minY': 23.489999771118164, 'maxY': 37.71500015258789} 
 {'minZ': -3.621000051498413, 'maxZ': 9.302000045776367}


*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

In [12]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(P,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('ts.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

results=Chem.SDMolSupplier('HG3_dock.sdf')


p=Chem.MolToMolBlock(results[0])

print('Reference: Magenta | LeDock Pose: Cyan')
print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

Reference: Magenta | LeDock Pose: Cyan
Pose: 8 | Score: -4.12


*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 5)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 6)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
fix_protein(filename=P,addHs_pH=7,try_renumberResidues=True,output='HG3_fix.pdb')
# load protein
prot = mda.Universe("HG3_fix.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

# load ligands
path = str('HG3_dock.sdf')
lig_suppl = list(plf.sdf_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
results_df = fp.to_dataframe(return_atoms=True)
results_df

  0%|          | 0/3 [00:00<?, ?it/s]

ligand               UNL1                                          
protein          ASP126.A       GLU46.A       GLY83.A       PRO45.A
interaction      Cationic      Cationic      Cationic      Cationic
Frame                                                              
0                  (4, 0)        (4, 0)        (4, 0)        (4, 0)
1            (None, None)  (None, None)  (None, None)  (None, None)
2            (None, None)  (None, None)  (None, None)  (None, None)

In [14]:
net = LigNetwork.from_ifp(results_df,lig_suppl[0],kind="frame", frame=0,rotation=270)
net.save('HG3_interaccions.html')
net.display()

In [15]:
P ='HG3.17_H.pdb'

cmd.load(filename=P,format='pdb',object='prot')
cmd.load(filename='ts.mol2',format='mol2',object='lig')

X,Y,Z= getbox(selection='lig',extending=5.0,software='ledock')
cmd.delete('all')

print(X,'\n',Y,'\n',Z)

generate_ledock_file(receptor=P,x=[X['minX'],X['maxX']],
                     y=[Y['minY'],Y['maxY']],
                     z=[Z['minZ'],Z['maxZ']],
                     n_poses=10,
                     rmsd=1.0,
                     l_list='lig.mol2', 
                     l_list_outfile='ledock_ligand.list',
                     out='dock.in')

!../bin/ledock_linux_x86 {'dock.in'}

dok_to_sdf(dok_file='lig.dok',output='HG3.17_dock.sdf')

{'minX': -15.470999717712402, 'maxX': 0.8289999961853027} 
 {'minY': 23.489999771118164, 'maxY': 37.71500015258789} 
 {'minZ': -3.621000051498413, 'maxZ': 9.302000045776367}


*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

In [16]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(P,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('ts.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

results=Chem.SDMolSupplier('HG3.17_dock.sdf')


p=Chem.MolToMolBlock(results[0])

print('Reference: Magenta | LeDock Pose: Cyan')
print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

Reference: Magenta | LeDock Pose: Cyan
Pose: 10 | Score: -4.86


ication,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 15)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 16)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
fix_protein(filename=P,addHs_pH=7,try_renumberResidues=True,output='HG3.17_fix.pdb')
# load protein
prot = mda.Universe("HG3.17_fix.pdb", guess_bonds=True)
prot = plf.Molecule.from_mda(prot)
prot.n_residues

# load ligands
path = str('HG3.17_dock.sdf')
lig_suppl = list(plf.sdf_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
results_df = fp.to_dataframe(return_atoms=True)
results_df

  0%|          | 0/1 [00:00<?, ?it/s]

ligand             UNL1                                                  \
protein         ALA21.A    ASP126.A     GLU46.A     GLY83.A    LEU235.A   
interaction Hydrophobic Hydrophobic Hydrophobic Hydrophobic Hydrophobic   
Frame                                                                     
0                (8, 4)     (11, 4)      (9, 2)     (11, 2)      (5, 9)   

ligand                                                               \
protein        MET171.A   MET236.A                MET266.A  TRP44.A   
interaction Hydrophobic HBAcceptor Hydrophobic Hydrophobic CationPi   
Frame                                                                 
0               (5, 11)     (2, 1)      (7, 7)     (9, 11)   (4, 7)   

ligand                   
protein                  
interaction Hydrophobic  
Frame                    
0               (5, 14)

In [18]:
net = LigNetwork.from_ifp(results_df,lig_suppl[0],kind="frame", frame=0,rotation=270)
net.save('HG3.17_interaccions.html')
net.display()